In [1]:
import pandas as pd
import numpy as np

In [4]:
users = pd.read_csv('./dataset/users.dat', sep='::', header=None, engine='python')
movies = pd.read_csv('./dataset/movies.dat', sep='::', header=None, engine='python', encoding = "ISO-8859-1")
ratings_test = pd.read_csv('./dataset/ratings_test.dat', sep='::', header=None, engine='python')
ratings_train = pd.read_csv('./dataset/ratings_train.dat', sep='::', header=None, engine='python')

In [5]:
users = users.rename(columns={
    0 : 'UserID',
    1 : 'Gender',
    2 : 'Age',
    3 : 'Occupation',
    4 : 'Zip-code'
})

In [6]:
movies = movies.rename(columns={
    0 : 'MovieID',
    1 : 'Title',
    2 : 'Genres'
})

In [7]:
ratings_test = ratings_test.rename(columns={
    0 : 'UserID',
    1 : 'MovieID',
    2 : 'Rating',
    3 : 'Timestamp'
})

In [8]:
ratings_train = ratings_train.rename(columns={
    0 : 'UserID',
    1 : 'MovieID',
    2 : 'Rating',
    3 : 'Timestamp'
})

In [9]:
user_movie_matrix = ratings_train.pivot(index = 'MovieID', columns = 'UserID', values = 'Rating')
user_movie_matrix = user_movie_matrix.T

In [10]:
user_movie_matrix_orig = user_movie_matrix

In [23]:
user_movie_matrix = (user_movie_matrix - np.nanmean(user_movie_matrix, axis=1).reshape(-1, 1)) / np.nanstd(user_movie_matrix, axis=1).reshape(-1, 1)
user_movie_matrix.fillna(0, inplace=True)

In [24]:
svd_decomposition = np.linalg.svd(user_movie_matrix.to_numpy())

In [32]:
factors_number = 20
u = svd_decomposition[0][:, :factors_number]
e = np.diag(svd_decomposition[1][:factors_number])
v = svd_decomposition[2][:factors_number, :]

In [33]:
r2 = np.matmul(u, e)
r2 = np.matmul(r2, v)
r2 = pd.DataFrame(r2)

r2.columns = user_movie_matrix_orig.columns
r2.index = user_movie_matrix_orig.index

r2 = r2 * np.nanstd(user_movie_matrix_orig, axis=1).reshape(-1, 1) + np.nanmean(user_movie_matrix_orig, axis=1).reshape(-1, 1)
r2[r2 > 5] = 5
r2[r2 < 1] = 1

rmse = 0
for i in range(ratings_test.shape[0]):
    user_id = ratings_test.iloc[i]['UserID']
    movie_id = ratings_test.iloc[i]['MovieID']
    rating = ratings_test.iloc[i]['Rating']
    rmse += (r2.loc[user_id][movie_id] - rating) ** 2
rmse = (rmse / ratings_test.shape[0]) ** 0.5
print(rmse)

rmse = 0
for i in range(ratings_train.shape[0]):
    user_id = ratings_train.iloc[i]['UserID']
    movie_id = ratings_train.iloc[i]['MovieID']
    rating = ratings_train.iloc[i]['Rating']
    rmse += (r2.loc[user_id][movie_id] - rating) ** 2
rmse = (rmse / ratings_train.shape[0]) ** 0.5
print(rmse)

0.9491200248419471
0.906437499546492


In [29]:
n = 50
sum(svd_decomposition[1][:n]) / sum(svd_decomposition[1])

0.07299213033265774